In [ ]:
"""
We aim to find the best assignment of host cities for the NCAA
tournament. We have a list of participating colleges and the distances
between them. We want to optimize for distance traveled and for rankings.
"""

In [ ]:
# importing libraries
using CSV, DataFrames, JuMP, Gurobi;

In [ ]:
# getting data

distances = CSV.read("distances.csv", DataFrame, header=false)

# rankings, get last column
rankings = CSV.read("rankings.csv", DataFrame, header=true)[:,5]

colleges_iter = 1:64

In [ ]:
distances = Matrix(distances)
rankings = Vector(rankings)

In [ ]:
"""
We compute normalized ranking scores as
1 - (team rank) / (total number of teams)
"""

ranking_scores = 1 .- rankings ./ 64;

"""
We compute normalized distance scores as
1 - (distance) / (maximum distance)
"""

distance_scores = 1 .- distances ./ maximum(distances);

In [ ]:
# creating model and decision variables
model = Model(Gurobi.Optimizer)
# make it not print out all the iterations
set_optimizer_attribute(model, "OutputFlag", 0)


# s_ij = 1 if college i hosts college j, 0 otherwise
@variable(model, s[colleges_iter, colleges_iter], Bin)



In [ ]:
# defining objective function

# weight we give to rankings
RW = 0.5
# weight we give to distances
DW = 0.5

@objective(model, Max, RW * sum(ranking_scores[i] * s[i,i] for i in colleges_iter) + DW * sum(distance_scores[i,j] * s[i,j] for i in colleges_iter, j in colleges_iter))

In [ ]:
# defining constraints

# each college can only go to one host
@constraint(model, [j in colleges_iter], sum(s[i,j] for i in colleges_iter) == 1)

# there are 8 host colleges
@constraint(model, sum(s[i,i] for i in colleges_iter) == 8)

# each hosting college hosts 8 colleges, including itself (only if s[i,i] = 1)
@constraint(model, [i in colleges_iter], sum(s[i,j] for j in colleges_iter) == 8 * s[i,i])



In [ ]:
optimize!(model)

@show objective_value(model)

In [ ]:
# printing host cities
for i in colleges_iter
    if value(s[i,i]) == 1
        println("College ", i, " is a host city.")
    end
end

In [ ]:
# now we test various values of RW and DW to see how the objective value changes and which cities are picked
function test_RW_DW(RW, DW)
    @objective(model, Max, RW * sum(ranking_scores[i] * s[i,i] for i in colleges_iter) + DW * sum(distance_scores[i,j] * s[i,j] for i in colleges_iter, j in colleges_iter))
    optimize!(model)
    println("RW = ", RW, ", DW = ", DW, ", objective value = ", objective_value(model))
    # ranking score
    println("Ranking score = ", sum(ranking_scores[i] * value(s[i,i]) for i in colleges_iter))
    # distance score
    println("Distance score = ", sum(distance_scores[i,j] * value(s[i,j]) for i in colleges_iter, j in colleges_iter))
    for i in colleges_iter
        if value(s[i,i]) == 1
            println("College ", i, " is a host city.")
        end
    end
end

In [ ]:
for RW in 0:0.1:1
    DW = 1 - RW
    test_RW_DW(RW, DW)
end